This is a quick re-work of the [`Keras` record transform example](http://winvector.github.io/FluidData/FluidDataReshapingWithCdata.html) in `Python`. For an `R` version please see [here](https://github.com/WinVector/cdata/blob/master/Examples/Inverse/Inverse.md).


In the [original article](http://winvector.github.io/FluidData/FluidDataReshapingWithCdata.html) we had `Keras` model performance data, which looked like the following.

In [1]:
import pandas
import data_algebra.cdata
import data_algebra.cdata_impl


df = pandas.DataFrame({
    'val_loss': [-0.377, -0.2997, -0.2964, -0.2779, -0.2843, -0.312],
    'val_acc': [0.8722, 0.8895, 0.8822, 0.8899, 0.8861, 0.8817],
    'loss': [-0.5067, -0.3002, -0.2166, -0.1739, -0.1411, -0.1136],
    'acc': [0.7852, 0.904, 0.9303, 0.9428, 0.9545, 0.9656],
    'epoch': [1, 2, 3, 4, 5, 6],
    })

df

,val_loss,val_acc,loss,acc,epoch
0,-0.3770,0.8722,-0.5067,0.7852,1
1,-0.2997,0.8895,-0.3002,0.9040,2
2,-0.2964,0.8822,-0.2166,0.9303,3
3,-0.2779,0.8899,-0.1739,0.9428,4
4,-0.2843,0.8861,-0.1411,0.9545,5
5,-0.3120,0.8817,-0.1136,0.9656,6



But for plotting, it is more convenient to have the data in the following form:

| epoch | measure                    | training | validation |
| ----: | :------------------------- | -------: | ---------: |
|     1 | minus binary cross entropy | \-0.5067 |   \-0.3770 |
|     1 | accuracy                   |   0.7852 |     0.8722 |
| ...                                                        |

[The article](http://winvector.github.io/FluidData/FluidDataReshapingWithCdata.html) uses ideas similar to [these](https://winvector.github.io/cdata/articles/design.html) to visualize the desired record structure and then write down this visualization as a concrete data record example.

The principle is: if you have a visualization of the input and output, it is then trivial to marshal these into a graphical representation of the desired transform. And if you can't work out what the input and output look like, then you really are not quite ready to perform the transform.  Knowing what we want is the minimum requirement and with this methodology it is also all that is needed.


In [2]:
shape = pandas.DataFrame({
    'measure': ['minus binary cross entropy', 'accuracy'],
    'training': ['loss', 'acc'],
    'validation': ['val_loss', 'val_acc'],
    })

shape

,measure,training,validation
0,minus binary cross entropy,loss,val_loss
1,accuracy,acc,val_acc


This description of the desired record shape is easily transformed into a data transformation specification.

In [3]:
record_map = data_algebra.cdata_impl.RecordMap(
    blocks_out=data_algebra.cdata.RecordSpecification(
        control_table=shape,
        record_keys=['epoch']
    ),
)

record_map

data_algebra.cdata_impl.RecordMap(
    blocks_in=None,
    blocks_out=data_algebra.cdata.RecordSpecification(
    record_keys=['epoch'],
    control_table=pandas.DataFrame({
    'measure': ['minus binary cross entropy', 'accuracy'],
    'training': ['loss', 'acc'],
    'validation': ['val_loss', 'val_acc'],
    }),
    control_table_keys=['measure']))

In [4]:
print(str(record_map))

Transform row records of the form:
  record_keys: ['epoch']
 ['epoch', 'loss', 'acc', 'val_loss', 'val_acc']
to block records of structure:
RecordSpecification
   record_keys: ['epoch']
   control_table_keys: ['measure']
   control_table:
                         measure training validation
   0  minus binary cross entropy     loss   val_loss
   1                    accuracy      acc    val_acc



Just about any transfrom we want can be specified through `data_algebra.cdata_impl.RecordMap` by specifying the `blocks_in` and `blocks_out` shapes (leaving these as `None` specifies the corresponding shape is a row record or record that is entirely in a single row).

We can easily apply this transform to our data.

In [5]:
res = record_map.transform(df)

res

,epoch,measure,training,validation
0,1,accuracy,0.7852,0.8722
1,1,minus binary cross entropy,-0.5067,-0.3770
2,2,accuracy,0.9040,0.8895
3,2,minus binary cross entropy,-0.3002,-0.2997
4,3,accuracy,0.9303,0.8822
5,3,minus binary cross entropy,-0.2166,-0.2964
6,4,accuracy,0.9428,0.8899
7,4,minus binary cross entropy,-0.1739,-0.2779
8,5,accuracy,0.9545,0.8861
9,5,minus binary cross entropy,-0.1411,-0.2843


And it is simple to build an inverse transform.

In [6]:
inv = record_map.inverse()

print(str(inv))

Transform block records of structure:
RecordSpecification
   record_keys: ['epoch']
   control_table_keys: ['measure']
   control_table:
                         measure training validation
   0  minus binary cross entropy     loss   val_loss
   1                    accuracy      acc    val_acc
to row records of the form:
  record_keys: ['epoch']
 ['epoch', 'loss', 'acc', 'val_loss', 'val_acc']



And equally easy to apply this inverse transform to data.

In [7]:
inv.transform(res)

,epoch,loss,val_loss,acc,val_acc
0,1,-0.5067,-0.3770,0.7852,0.8722
1,2,-0.3002,-0.2997,0.9040,0.8895
2,3,-0.2166,-0.2964,0.9303,0.8822
3,4,-0.1739,-0.2779,0.9428,0.8899
4,5,-0.1411,-0.2843,0.9545,0.8861
5,6,-0.1136,-0.3120,0.9656,0.8817


Notice how each step can be inspected and checked as we worked. I would definitely recommend re-reading [the original article](http://winvector.github.io/FluidData/FluidDataReshapingWithCdata.html) with the new transform notation in mind. In any case, please check out the `cdata` [package](https://github.com/WinVector/cdata) and [documentation](https://winvector.github.io/cdata/).